In [1]:
#Import packages used below
import numpy as np
import pandas as pd
##pip3 install multiprocess
import multiprocessing as mp 
import matplotlib.pyplot as plt
import itertools
import copy as cp
import DeferredAcceptance as da
from IPython.display import display


import warnings
import time
import pickle

In [2]:

def demoFxn(n, sigma_fract):

    #define rank lists
    men_source = da.preflists(n, sigma_fract)
    women_source = da.preflists(n, sigma_fract)

    men_long = cp.deepcopy(men_source[2])
    men = cp.deepcopy(men_source[0])
    men_as_suiteds = cp.deepcopy(men_source[1])
    men_as_suiteds_long = cp.deepcopy(men_source[3])

    women = cp.deepcopy(women_source[1])
    women_long = cp.deepcopy(women_source[1])
    women_as_suitors = cp.deepcopy(women_source[0])
    women_as_suitors_long = cp.deepcopy(women_source[2])


    #matching
    MoSm_long = da.GSDA(men_long, women_long)
    #matching with women proposing
    #print(men_as_suiteds_long)
    #print(women_as_suitors_long)
    MoSm_WP_long = da.GSDA(women_as_suitors_long, men_as_suiteds_long)

    #print("MP")
    #print(MoSm_long)
    #print("WP")
    #print(MoSm_WP_long)
    #long_core_size
    core_same_long = 0
    core_diff_long = 0

    #initialize dataframe of rejection chains
    rejectionChainVars = ['Woman ID', 
                          'I. Match', 
                          'I. PrefRank', 
                          'N. Match', 
                          'N. PrefRank', 
                          'improvement', 
                          'I. RelRank', 
                          'N. RelRank', 
                          'Chain Length', 
                          'Longest Chain', 
                          'No. Involved Men']
    rejectionChains = pd.DataFrame([], columns=rejectionChainVars) 
    listDepthVars = ['Woman_ID', 'I_PrefRank', 'N_PrefRank', 'Color' ]
    listDepth = pd.DataFrame([], columns = listDepthVars)
    for k, v in MoSm_long.items():
        if MoSm_WP_long.get(women_as_suitors_long[v.id]).id == k.id :
            core_same_long += 1
            oldPrefRank = v.id - cp.deepcopy(women_long[v.id].prefList.index(women_long[v.id].held.id)) 
            listDepth  = listDepth.append(pd.DataFrame([[v.id, oldPrefRank, oldPrefRank, 'Blue']], columns = listDepthVars))
            #print(type(v.id))
        else:
            core_diff_long += 1
            #print("MP: woman " + str(v.id) + " and man " + str(k.id))

            ##See how long the rejection chain is:
            #first make new copy of the suiteds and sutors objects 
            #(note that these objects already have the number of rejections etc coded into them)
            men_long_cp = cp.deepcopy(men_long)
            women_long_cp = cp.deepcopy(women_long)

            #examine the woman:
            woman_id = cp.deepcopy(women_long_cp[v.id].id)
            woman_init_match = cp.deepcopy(women_long_cp[v.id].held.id)
            woman_init_prefRank = cp.deepcopy(women_long_cp[v.id].prefList.index(women_long_cp[v.id].held.id))
            #print("woman: "+ str(women_long_cp[v.id].id) + " held: "+ str(women_long_cp[v.id].held.id) +
            #      ", her " + str(women_long_cp[v.id].prefList.index(women_long_cp[v.id].held.id)) +"th preference")

            #truncate woman's pref list
            women_long_cp[v.id].prefList = women_long_cp[v.id].prefList[:women_long_cp[v.id].prefList.index(women_long_cp[v.id].held.id)]


            #move "held" man to "current suitors"
            women_long_cp[v.id].currentSuitors |= set([women_long_cp[v.id].held])
            women_long_cp[v.id].held = None

            #reject man/setup resumed DA
            unassigned = set()
            for suited in women_long_cp:
                unassigned |= suited.reject()
            for suitor in unassigned:
                suitor.rejections += 1


            #run DA, counting steps
            rejectionChain = 1
            longestRejectionChain = 1
            newRejected = cp.copy(unassigned)
            while len(unassigned) > 0:
                if unassigned - newRejected != set():
                    longestRejectionChain = 0
                newRejected |= unassigned
                for suitor in unassigned:
                    next_proposal = suitor.preference()
                    if next_proposal != None:
                        women_long_cp[suitor.preference()].currentSuitors.add(suitor)
                unassigned = set()
                for suited in women_long_cp:
                    unassigned |= suited.reject()

                for suitor in unassigned:
                    suitor.rejections += 1

                rejectionChain += 1
                longestRejectionChain += 1

            woman_new_match = cp.deepcopy(women_long_cp[v.id].held.id)
            woman_new_prefRank = cp.deepcopy(women_long_cp[v.id].prefList.index(women_long_cp[v.id].held.id))
            #print("is now matched with man " 
            #      + str(women_long_cp[v.id].held.id)+ ", her " + 
            #      str(women_long_cp[v.id].prefList.index(women_long_cp[v.id].held.id)) 
            #      +"th preference")
            #print("Rejection Chain Length " +  str(rejectionChain))                      
            #print(" ")

            rejectionChains = rejectionChains.append(pd.DataFrame([[
                woman_id, 
                woman_init_match , 
                woman_init_prefRank, 
                woman_new_match, 
                woman_new_prefRank, 
                woman_init_prefRank - woman_new_prefRank, 
                woman_id - woman_init_prefRank,
                woman_id - woman_new_prefRank, 
                rejectionChain,
                longestRejectionChain,
                len(newRejected)
                ]], columns=rejectionChainVars) )

    assert core_same_long + core_diff_long == n
    core_size_long = core_diff_long/ n

    print("Core Difference: " + str(core_diff_long))

    list_depth = pd.DataFrame(men_long, columns=['Men'])
    def man_as_id(row):
        return(row['Men'].id)
    def ranklist_depth(row):
        man = row['Men']
        manid = row['Men'].id
        man_got = MoSm_long.get(man).id
        #print(type(man_got))
        man_got_rank = man.prefList.index(man_got)
        return(man_got_rank - manid)


    listDepth = listDepth.astype({listDepthVars[0]: 'int32',listDepthVars[1]: 'int32',listDepthVars[2]: 'int32'})
    rejectionChains = rejectionChains.astype('int32')
    nm_1 = listDepth.plot(kind='scatter', x = 'Woman_ID',  y = 'N_PrefRank')
    im_1 = listDepth.plot(kind='scatter', x = 'Woman_ID',  y = 'I_PrefRank' ,  ax = nm_1)
    if len(rejectionChains) > 0 :
        nm_2 = rejectionChains.plot(kind='scatter', x = 'Woman ID',  y =  'N. RelRank'  , c = "r" ,ax = im_1)
        im_2 = rejectionChains.plot(kind='scatter', x = 'Woman ID',  y =  'I. RelRank'  , c = "r" ,ax = nm_2)

        print(rejectionChains.plot(kind = 'scatter', x = 'Longest Chain', y = 'No. Involved Men'))
        print(im_2)
    display(rejectionChains)

In [3]:
demoFxn(100, .001)


<class 'DeferredAcceptance.Suitor'>
0
<class 'DeferredAcceptance.Suitor'>
1
<class 'DeferredAcceptance.Suitor'>
2
<class 'DeferredAcceptance.Suitor'>
3
<class 'DeferredAcceptance.Suitor'>
4
<class 'DeferredAcceptance.Suitor'>
5
<class 'DeferredAcceptance.Suitor'>
6
<class 'DeferredAcceptance.Suitor'>
7
<class 'DeferredAcceptance.Suitor'>
8
<class 'DeferredAcceptance.Suitor'>
9
<class 'DeferredAcceptance.Suitor'>
10
<class 'DeferredAcceptance.Suitor'>
11
<class 'DeferredAcceptance.Suitor'>
12
<class 'DeferredAcceptance.Suitor'>
13
<class 'DeferredAcceptance.Suitor'>
14
<class 'DeferredAcceptance.Suitor'>
15
<class 'DeferredAcceptance.Suitor'>
16
<class 'DeferredAcceptance.Suitor'>
17
<class 'DeferredAcceptance.Suitor'>
18
<class 'DeferredAcceptance.Suitor'>
19
<class 'DeferredAcceptance.Suitor'>
20
<class 'DeferredAcceptance.Suitor'>
21
<class 'DeferredAcceptance.Suitor'>
22
<class 'DeferredAcceptance.Suitor'>
23
<class 'DeferredAcceptance.Suitor'>
24
<class 'DeferredAcceptance.Suitor'>

28is assigned_count: 1
29th's assigned_count 0, proposal number 4
29th's assigned_count 0, proposal number 4
29is assigned_count: 1
30th's assigned_count 0, proposal number 4
30th's assigned_count 0, proposal number 4
30is assigned_count: 1
31th's assigned_count 0, proposal number 4
31th's assigned_count 0, proposal number 4
31is assigned_count: 1
32th's assigned_count 0, proposal number 4
32th's assigned_count 0, proposal number 4
32is assigned_count: 1
33th's assigned_count 0, proposal number 4
33th's assigned_count 0, proposal number 4
33is assigned_count: 1
34th's assigned_count 0, proposal number 4
34th's assigned_count 0, proposal number 4
34is assigned_count: 1
35th's assigned_count 0, proposal number 4
35th's assigned_count 0, proposal number 4
35is assigned_count: 1
36th's assigned_count 0, proposal number 4
36th's assigned_count 0, proposal number 4
36is assigned_count: 1
37th's assigned_count 0, proposal number 4
37th's assigned_count 0, proposal number 4
37is assigned_count

42th's assigned_count 0, proposal number 9
42is assigned_count: 1
78th's assigned_count 0, proposal number 9
78th's assigned_count 0, proposal number 9
78is assigned_count: 1
43th's assigned_count 0, proposal number 9
43th's assigned_count 0, proposal number 9
43is assigned_count: 1
79th's assigned_count 0, proposal number 9
79th's assigned_count 0, proposal number 9
79is assigned_count: 1
44th's assigned_count 0, proposal number 9
44th's assigned_count 0, proposal number 9
44is assigned_count: 1
80th's assigned_count 0, proposal number 9
80th's assigned_count 0, proposal number 9
80is assigned_count: 1
45th's assigned_count 0, proposal number 9
45th's assigned_count 0, proposal number 9
45is assigned_count: 1
81th's assigned_count 0, proposal number 9
81th's assigned_count 0, proposal number 9
81is assigned_count: 1
46th's assigned_count 0, proposal number 9
46th's assigned_count 0, proposal number 9
46is assigned_count: 1
82th's assigned_count 0, proposal number 9
82th's assigned_cou

38is assigned_count: 1
74th's assigned_count 0, proposal number 14
74th's assigned_count 0, proposal number 14
74is assigned_count: 1
39th's assigned_count 0, proposal number 14
39th's assigned_count 0, proposal number 14
39is assigned_count: 1
75th's assigned_count 0, proposal number 14
75th's assigned_count 0, proposal number 14
75is assigned_count: 1
40th's assigned_count 0, proposal number 14
40th's assigned_count 0, proposal number 14
40is assigned_count: 1
76th's assigned_count 0, proposal number 14
76th's assigned_count 0, proposal number 14
76is assigned_count: 1
41th's assigned_count 0, proposal number 14
41th's assigned_count 0, proposal number 14
41is assigned_count: 1
77th's assigned_count 0, proposal number 14
77th's assigned_count 0, proposal number 14
77is assigned_count: 1
42th's assigned_count 0, proposal number 14
42th's assigned_count 0, proposal number 14
42is assigned_count: 1
78th's assigned_count 0, proposal number 14
78th's assigned_count 0, proposal number 14
7

10th's assigned_count 0, proposal number 20
10th's assigned_count 0, proposal number 20
10is assigned_count: 1
11th's assigned_count 0, proposal number 20
11th's assigned_count 0, proposal number 20
11is assigned_count: 1
12th's assigned_count 0, proposal number 20
12th's assigned_count 0, proposal number 20
12is assigned_count: 1
13th's assigned_count 0, proposal number 20
13th's assigned_count 0, proposal number 20
13is assigned_count: 1
14th's assigned_count 0, proposal number 20
14th's assigned_count 0, proposal number 20
14is assigned_count: 1
15th's assigned_count 0, proposal number 20
15th's assigned_count 0, proposal number 20
15is assigned_count: 1
16th's assigned_count 0, proposal number 20
16th's assigned_count 0, proposal number 20
16is assigned_count: 1
17th's assigned_count 0, proposal number 20
17th's assigned_count 0, proposal number 20
17is assigned_count: 1
18th's assigned_count 0, proposal number 20
18th's assigned_count 0, proposal number 20
18is assigned_count: 1
1

...

34th's assigned_count 0, proposal number 76
34is assigned_count: 1
35th's assigned_count 0, proposal number 76
35th's assigned_count 0, proposal number 76
35is assigned_count: 1
36th's assigned_count 0, proposal number 76
36th's assigned_count 0, proposal number 76
36is assigned_count: 1
37th's assigned_count 0, proposal number 76
37th's assigned_count 0, proposal number 76
37is assigned_count: 1
73th's assigned_count 0, proposal number 76
73th's assigned_count 0, proposal number 76
73is assigned_count: 1
38th's assigned_count 0, proposal number 76
38th's assigned_count 0, proposal number 76
38is assigned_count: 1
74th's assigned_count 0, proposal number 76
74th's assigned_count 0, proposal number 76
74is assigned_count: 1
39th's assigned_count 0, proposal number 76
39th's assigned_count 0, proposal number 76
39is assigned_count: 1
75th's assigned_count 0, proposal number 76
75th's assigned_count 0, proposal number 76
75is assigned_count: 1
40th's assigned_count 0, proposal number 76
4

70th's assigned_count 0, proposal number 81
70th's assigned_count 0, proposal number 81
70is assigned_count: 1
71th's assigned_count 0, proposal number 81
71th's assigned_count 0, proposal number 81
71is assigned_count: 1
72th's assigned_count 0, proposal number 81
72th's assigned_count 0, proposal number 81
72is assigned_count: 1
16th's assigned_count 2, proposal number 82
16th's assigned_count 2, proposal number 82
16is assigned_count: 3
17th's assigned_count 0, proposal number 82
17th's assigned_count 0, proposal number 82
17is assigned_count: 1
18th's assigned_count 0, proposal number 82
18th's assigned_count 0, proposal number 82
18is assigned_count: 1
19th's assigned_count 0, proposal number 82
19th's assigned_count 0, proposal number 82
19is assigned_count: 1
20th's assigned_count 0, proposal number 82
20th's assigned_count 0, proposal number 82
20is assigned_count: 1
21th's assigned_count 0, proposal number 82
21th's assigned_count 0, proposal number 82
21is assigned_count: 1
2

94is assigned_count: 1
59th's assigned_count 0, proposal number 88
59th's assigned_count 0, proposal number 88
59is assigned_count: 1
95th's assigned_count 0, proposal number 88
95th's assigned_count 0, proposal number 88
95is assigned_count: 1
60th's assigned_count 0, proposal number 88
60th's assigned_count 0, proposal number 88
60is assigned_count: 1
96th's assigned_count 0, proposal number 88
96th's assigned_count 0, proposal number 88
96is assigned_count: 1
61th's assigned_count 0, proposal number 88
61th's assigned_count 0, proposal number 88
61is assigned_count: 1
97th's assigned_count 0, proposal number 88
97th's assigned_count 0, proposal number 88
97is assigned_count: 1
62th's assigned_count 0, proposal number 88
62th's assigned_count 0, proposal number 88
62is assigned_count: 1
98th's assigned_count 0, proposal number 88
98th's assigned_count 0, proposal number 88
98is assigned_count: 1
63th's assigned_count 0, proposal number 88
63th's assigned_count 0, proposal number 88
6

86th's assigned_count 0, proposal number 96
86th's assigned_count 0, proposal number 96
86is assigned_count: 1
51th's assigned_count 0, proposal number 96
51th's assigned_count 0, proposal number 96
51is assigned_count: 1
87th's assigned_count 0, proposal number 96
87th's assigned_count 0, proposal number 96
87is assigned_count: 1
52th's assigned_count 0, proposal number 96
52th's assigned_count 0, proposal number 96
52is assigned_count: 1
88th's assigned_count 0, proposal number 96
88th's assigned_count 0, proposal number 96
88is assigned_count: 1
89th's assigned_count 0, proposal number 96
89th's assigned_count 0, proposal number 96
89is assigned_count: 1
53th's assigned_count 0, proposal number 96
53th's assigned_count 0, proposal number 96
53is assigned_count: 1
90th's assigned_count 0, proposal number 96
90th's assigned_count 0, proposal number 96
90is assigned_count: 1
54th's assigned_count 0, proposal number 96
54th's assigned_count 0, proposal number 96
54is assigned_count: 1
5

IndexError: tuple index out of range